# Introduction -- Reminder

Each Python lab will start with a pre-flight exercise that walks through building some of the set up and tools ($\sim$ 30 min), followed by an in-class tutorial with time for Q+A (50 min) so you can walk through steps that will be necessary for the homework assignment you will submit ($\sim$ 3 hrs).  Each lab will contain starter code, similar to what you see below.  Please fill in the code to complete the pre-flight assignment in preparation for the in-class tutorial.  

Preflight ($\sim$30-60 min, 10 points) **Typically due: Wednesdays 3pm EST**

*Preflight typically graded by Wednesday 5p EST*

In-class tutorial and Q+A ($\sim$ 50 min, 10 points) **Typically occurs: Thursdays 9am EST**

Homework assignment ($\sim$ 3-5 hrs, 30 points) **Typically due: Mondays 9am EST** 

*Homework typically graded by Thursday 5p*

When we grade your homework, we will not run your code. Once submitted, your notebook should have the outputs for all of your results.  Please do not include long outputs from debugging, beyond a few print statements and the requested visualizations (i.e. plots).

**Grading:** When we grade your notebook, we will convert the .ipynb file to an HTML file.  We will be using [nbgrader](https://nbgrader.readthedocs.io/en/stable/) to grade your notebooks.  **Note:** If your ```IPython``` version is too old, we will *not* be able to grade your assignments.


# Phys 260 Python Lab 2: More Python: Tutorial (20 points total)

## Tutorial summary
- Discussion of meshgrid inputs (args and kwargs) and outputs (returned values/objects). 
- Efield calculation for more than one point with `np.apply_along_axis` (two point example).
- Discretize a charge distribution (sphere example, exercise with cube example)
- Build a function to take arbitrary points, arbitrary charges, and arbitrary `points_in_meshgrid`. 



Task:  Define dq
`calculate_efield_due_to_points(charge_points, points_in_meshgrid, dq)`
- Task in calculate efield, have them fill out the sum portion.
- Make ONE (or two) of the 1-d plots, talk about it (Gauss vs. Computer)

In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [14]:
#  Here we set up the points in our vector field
numpoints_1d = 10

sample_points = np.linspace(-5,5,num=numpoints_1d)

xarray, yarray, zarray = np.meshgrid(sample_points,
                                     sample_points,
                                     sample_points,
                                     indexing='ij'
                                    )

points_in_meshgrid = np.array([xarray, yarray, zarray])
sample_points.shape

(10,)

## Brief review (3 points)
**Quick questions for the class** (class discussion -- 10 min 3 points): 
- What do the first three arguments of `np.meshgrid` do?
- What happens when we switch indexing to 'xy' (or if we leave out the kwarg indexing altogether)?
- What shape should `sample_points`, `xarray`, `yarray`, `zarray`, and points_in_meshgrid have?

- The first three arguments correspond to the extend and one dimensional sampled points across each axis.
- When we switch indexing to 'xy' (equivalent of leaving out the kwarg because 'xy' is the default), then the first and second dimensions switch.
- The shapes should be `(numpoints_1d,)`; `(numpoints_1d, numpoints_1d, numpoints_1d)` for all x, y, z arrays;  `points_in_meshgrid` should be `(3,numpoints_1d, numpoints_1d, numpoints_1d)`, 

## The efield calculator

In [9]:
def calculate_efield_at_point(charge_position, field_position, q=1) :
    '''Return the electric field due to a point charge.
    
    Inputs:
    charge_position (n-darray) : x, y, and z position vector
    field_positions_x (n-darray) : x coordinates of the field
    field_positions_y (n-darray) : y coordinates of the field
    field_positions_z (n-darray) : z coordinates of the field
    q (float or int) : charge of the point
    Outputs:
    vector_efield (n-darray) : x, y, z components of the e-field at the point field_position
    '''
    
    r = field_position-charge_position
    r_magnitude = np.linalg.norm(field_position-charge_position)
    r_unit = r/r_magnitude
    
    k = 1  #  To make plotting extents easier to deal with
    
    return k*q / r_magnitude**2 * r_unit

## Finding the e-field due to a point charge

Let us recall the one line calculation for a point charge.

In [7]:
charge_position = np.array([0,0,0])
efield_vectors_for_point_charge = np.apply_along_axis(calculate_efield_at_point, 0, 
                                                         points_in_meshgrid, charge_position)

## Find the e-field due to two point charges (3 points)

**Group Exercise** (10min -- 3 points): Assume that you have a point charge at (1,0,0) and at (-1,0,0) each with charge 0.5C.   Calculate the `efield_vectors_for_two_charges` below.  This is similar to the dipole calculation you did in the first tutorial/hw.
- You will want to use the function `calculate_efield_at_point`.
- First, you will need to incorporate the keyword argument `q=0.5`.
- Use the principle of superposition here.
- Aim to do this in 2-5 lines, but your steps should be clear.

In [13]:
# Calculate efield_vectors_for_two_charges here
### BEGIN SOLUTION
efield_from_1 = np.apply_along_axis(calculate_efield_at_point, 0, 
                                                points_in_meshgrid, np.array([1,0,0]), q=0.5)
efield_from_2 = np.apply_along_axis(calculate_efield_at_point, 0, 
                                                points_in_meshgrid, np.array([-1,0,0]), q=0.5)
efield_vectors_for_two_charges = efield_from_1+efield_from_2
### END SOLUTION
assert(efield_vectors_for_two_charges.shape == (3, numpoints_1d, numpoints_1d, numpoints_1d))

## Discretizing a charge distribution (3 points)

Let's say that instead of a point charge, we now have a charge distribution.  For initial simplicity, let us consider an origin centered insulating sphere, total charge $Q=1$C, with radius 0.75.

Now, do the same for an origin centered cube, total charge $Q=1$C, whose volume lies within $-2\leq x,y,z\leq 2$..

## Vectorized finding distance between points (1 point)

We can also find the distance between a point, and a series of points (the series of points being the points generated from `np.meshgrid`, xarray, yarray, and zarray.  In the example below, you'll notice that we had to take the transpose to properly subtract each x, y, z meshgrid point from the `example_point` x, y, and z.

In [1]:
example_point = np.array([0,0,0])
points_in_meshgrid_t = points_in_meshgrid.T
print(points_in_meshgrid_t.shape)

# Distance between origin and each point
distance_all_points = np.linalg.norm(points_in_meshgrid_t - example_point, axis=3)

# There is a distance value for each point, 
# so the shape of distance_all_points is the same as 
#  xarray (or yarray or zarray):
print(distance_all_points.shape)


NameError: name 'np' is not defined

**Try:** Taking out the `.T` in the second line and re-executing the cell.  You'll notice that you get a value error because the origin has shape (3,) and the points_in_meshgrid has shape (3,4,4,4).  The way numpy matches up the operation is such that we need to have something with (4,4,4,3).  (Think about the difference in taking matrix multiplication of an mxn with an nxl matrix versus mxn and lxn.  One of these you cannot do for m!=n!=l.)  

Below is the skeleton for a new function `calculate_efield_from_charge` that is very analogous to the `calculate_efield_at_point` from tutorial/hw 1.  

Use the example code in the code cell above to fill out the function below. **In the function you fill out below**, you'll need to:


- First, define `points_in_efield_meshgrid` given `field_positions_x`, `field_positions_y`, and `field_positions_z` (these are the analogs to `xarray`, `yarray`, and `zarray` when we defined `points_in_meshgrid` a few cells above.


- Second, calculate `r_magnitude` for all points (should take one line of code).  `r_magnitude` in the function below is the analog to `distance_all_points` in the code cell above, and `charge_position` is the analog to `example_point` in the code cell above.

# Phys 260 Python Lab 2: More Python: Homework (10 points total)


Your homework assignment begins here.  Note, the homework below relies on code built during the tutorial (above)

## Homework summary
- Key Task: Define charge points, and dq changes depending on number of points